# DX 704 Week 8 Project

This homework will modify a simulator controlling a small vehicle to implement tabular q-learning.
You will first test your code with random and greedy-epsilon policies, then tweak your own training method for a more optimal policy.

The full project description and a template notebook are available on GitHub: [Project 8 Materials](https://github.com/bu-cds-dx704/dx704-project-08).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Rover Simulator

The following Python class implements a simulation of a simple vehicle with integer x,y coordinates facing in one of 8 possible directions.


In [1]:
# DO NOT CHANGE

import random

class RoverSimulator(object):
    DIRECTIONS = ((0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1), (-1, 0), (-1, 1))

    def __init__(self, resolution):
        self.resolution = resolution
        self.terminal_state = self.construct_state(resolution // 2, resolution // 2, 0)

        self.initial_states = []
        for initial_x in (0, resolution // 2, resolution - 1):
            for initial_y in (0, resolution // 2, resolution - 1):
                for initial_direction in range(8):
                    initial_state = self.construct_state(initial_x, initial_y, initial_direction)
                    if initial_state != self.terminal_state:
                        self.initial_states.append(initial_state)

    def construct_state(self, x, y, direction):
        assert 0 <= x < self.resolution
        assert 0 <= y < self.resolution
        assert 0 <= direction < 8

        state = (y * self.resolution + x) * 8 + direction
        assert self.decode_state(state) == (x, y, direction)
        return state

    def decode_state(self, state):
        direction = state % 8
        x = (state // 8) % self.resolution
        y = state // (8 * self.resolution)

        return (x, y, direction)

    def get_actions(self, state):
        return [-1, 0, 1]

    def get_next_reward_state(self, curr_state, curr_action):
        if curr_state == self.terminal_state:
            # no rewards or changes from terminal state
            return (0, curr_state)

        (curr_x, curr_y, curr_direction) = self.decode_state(curr_state)
        (curr_dx, curr_dy) = self.DIRECTIONS[curr_direction]

        assert self.construct_state(curr_x, curr_y, curr_direction) == curr_state

        assert curr_action in (-1, 0, 1)

        next_x = min(max(0, curr_x + curr_dx), self.resolution - 1)
        next_y = min(max(0, curr_y + curr_dy), self.resolution - 1)
        next_direction = (curr_direction + curr_action) % 8

        next_state = self.construct_state(next_x, next_y, next_direction)
        next_reward = 1 if next_state == self.terminal_state else 0

        return (next_reward, next_state)

    def rollout_policy(self, policy_func, max_steps=1000):
        curr_state = self.sample_initial_state()
        for i in range(max_steps):
            curr_action = policy_func(curr_state, self.get_actions(curr_state))
            (next_reward, next_state) = self.get_next_reward_state(curr_state, curr_action)
            yield (curr_state, curr_action, next_reward, next_state)
            curr_state = next_state

    def sample_initial_state(self):
        return random.choice(self.initial_states)

In [2]:
simulator = RoverSimulator(16)
initial_sample = simulator.sample_initial_state()
print("INITIAL SAMPLE", initial_sample)

INITIAL SAMPLE 1984


## Part 1: Implement a Random Policy

Random policies are often used to test simulators and start initial exploration.
Implement a random policy for these simulators.

In [25]:
def random_policy(state, actions):
    return random.choice(actions)

Use the code below to test your random policy.
Then modify it to save the results in "log-random.tsv" with the columns curr_state, curr_action, next_reward and next_state.

In [28]:
import pandas as pd

results_1 = []
for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(random_policy, max_steps=32):
    print("CURR STATE", curr_state, "ACTION", curr_action, "NEXT REWARD", next_reward, "NEXT STATE", next_state)
    results_1.append({
        'curr_state': curr_state,
        'curr_action': curr_action,
        'next_reward': next_reward,
        'next_state': next_state
    })

CURR STATE 68 ACTION 0 NEXT REWARD 0 NEXT STATE 68
CURR STATE 68 ACTION -1 NEXT REWARD 0 NEXT STATE 67
CURR STATE 67 ACTION 1 NEXT REWARD 0 NEXT STATE 76
CURR STATE 76 ACTION 0 NEXT REWARD 0 NEXT STATE 76
CURR STATE 76 ACTION -1 NEXT REWARD 0 NEXT STATE 75
CURR STATE 75 ACTION 0 NEXT REWARD 0 NEXT STATE 83
CURR STATE 83 ACTION 1 NEXT REWARD 0 NEXT STATE 92
CURR STATE 92 ACTION -1 NEXT REWARD 0 NEXT STATE 91
CURR STATE 91 ACTION 0 NEXT REWARD 0 NEXT STATE 99
CURR STATE 99 ACTION -1 NEXT REWARD 0 NEXT STATE 106
CURR STATE 106 ACTION 1 NEXT REWARD 0 NEXT STATE 115
CURR STATE 115 ACTION -1 NEXT REWARD 0 NEXT STATE 122
CURR STATE 122 ACTION 1 NEXT REWARD 0 NEXT STATE 123
CURR STATE 123 ACTION 0 NEXT REWARD 0 NEXT STATE 123
CURR STATE 123 ACTION 0 NEXT REWARD 0 NEXT STATE 123
CURR STATE 123 ACTION -1 NEXT REWARD 0 NEXT STATE 122
CURR STATE 122 ACTION -1 NEXT REWARD 0 NEXT STATE 121
CURR STATE 121 ACTION -1 NEXT REWARD 0 NEXT STATE 248
CURR STATE 248 ACTION -1 NEXT REWARD 0 NEXT STATE 383
CUR

Submit "log-random.tsv" in Gradescope.

In [29]:
# Save to TSV file
df_1 = pd.DataFrame(results_1)
df_1.to_csv('submission/log-random.tsv', sep='\t', index=False)

## Part 2: Implement Q-Learning with Random Policy

The code below runs 32 random rollouts of 1024 steps using your random policy.
Modify the rollout code to implement Q-Learning.
Just implement one learning update for each sampled state-action in the simulation.
Use $\alpha=1$ and $\gamma=0.9$ since the simulator is deterministic and there is a sink where the rewards stop.




In [34]:
# Initialize Q-table as a dictionary
q_2 = {}

alpha = 1.0  # learning rate
gamma = 0.9  # discount factor

# Store all learning steps for the TSV file
q_log_2 = []

for episode in range(32):
    for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(random_policy, max_steps=1024):
        # Get current Q-value (default to 0 if not seen before)
        old_value = q_2.get((curr_state, curr_action), 0.0)
        
        # Calculate max Q-value for next state
        next_actions = simulator.get_actions(next_state)
        max_next_q = max([q_2.get((next_state, a), 0.0) for a in next_actions])

        # Q-learning update: Q(s,a) = Q(s,a) + alpha * [r + gamma * max_a' Q(s',a') - Q(s,a)]
        new_value = old_value + alpha * (next_reward + gamma * max_next_q - old_value)
        
        # Update Q-table
        q_2[(curr_state, curr_action)] = new_value
        
        # Log this step
        q_log_2.append({
            'curr_state': curr_state,
            'curr_action': curr_action,
            'next_reward': next_reward,
            'next_state': next_state,
            'old_value': old_value,
            'new_value': new_value
        })

Save each step in the simulator in a file "q-random.tsv" with columns curr_state, curr_action, next_reward, next_state, old_value, new_value.

In [35]:
# Save Q-learning log to TSV file
df_q_2 = pd.DataFrame(q_log_2)
df_q_2.to_csv('submission/q-random.tsv', sep='\t', index=False)

print(f"Saved {len(q_log_2)} Q-learning updates to q-random.tsv")
print(f"Learned Q-values for {len(q_2)} state-action pairs")

Saved 32768 Q-learning updates to q-random.tsv
Learned Q-values for 5274 state-action pairs


Submit "q-random.tsv" in Gradescope.

## Part 3: Implement Epsilon-Greedy Policy

Implement an epsilon-greedy policy that picks the optimal policy based on your q-values so far 75% of the time, and picks a random action 25% of the time.
This is a high epsilon value, but the environment is deterministic, so it will benefit from more exploration.

In [7]:
def epsilon_greedy_policy(state, actions):
    epsilon = 0.25
    
    # With probability epsilon, choose a random action (exploration)
    if random.random() < epsilon:
        return random.choice(actions)
    
    # Otherwise, choose the action with highest Q-value (exploitation)
    # If no Q-values exist yet, return random action
    q_values_for_state = [(action, q_3.get((state, action), 0.0)) for action in actions]
    best_action = max(q_values_for_state, key=lambda x: x[1])[0]
    
    return best_action

Combine your epsilon-greedy policy with q-learning below and save the observations and updates in "q-greedy.tsv" with columns curr_state, curr_action, next_reward, next_state, old_value, new_value.

Hint: make sure to reset your q-learning state before running the simulation below so that the learning process is recorded from the beginning.

In [36]:
# Reset Q-learning state
q_3 = {}

alpha = 1.0  # learning rate
gamma = 0.9  # discount factor

# Store all learning steps for the TSV file
q_log_3 = []

for episode in range(32):
    for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(epsilon_greedy_policy, max_steps=1024):
        # Get current Q-value (default to 0 if not seen before)
        old_value = q_3.get((curr_state, curr_action), 0.0)
        
        # Calculate max Q-value for next state
        next_actions = simulator.get_actions(next_state)
        max_next_q = max([q_3.get((next_state, a), 0.0) for a in next_actions])
        
        # Q-learning update: Q(s,a) = Q(s,a) + alpha * [r + gamma * max_a' Q(s',a') - Q(s,a)]
        new_value = old_value + alpha * (next_reward + gamma * max_next_q - old_value)
        
        # Update Q-table
        q_3[(curr_state, curr_action)] = new_value
        
        # Log this step
        q_log_3.append({
            'curr_state': curr_state,
            'curr_action': curr_action,
            'next_reward': next_reward,
            'next_state': next_state,
            'old_value': old_value,
            'new_value': new_value
        })

        if next_reward > 0:
            # moving to terminal state
            break

In [37]:
# Save Q-learning log to TSV file
df_q_3 = pd.DataFrame(q_log_3)
df_q_3.to_csv('submission/q-greedy.tsv', sep='\t', index=False)

print(f"Saved {len(q_log_3)} Q-learning updates to q-greedy.tsv")
print(f"Learned Q-values for {len(q_3)} state-action pairs")

Saved 32768 Q-learning updates to q-greedy.tsv
Learned Q-values for 170 state-action pairs


Submit "q-greedy.tsv" in Gradescope.

## Part 4: Extract Policy from Q-Values

Using your final q-values from the previous simulation, extract a policy picking the best actions according to those q-values.
Save the policy in a file "policy-greedy.tsv" with columns state and action.

In [43]:
# Extract policy from Q-values learned in Part 3
# For each state that appears in q_3, find the action with highest Q-value

policy_greedy = []

# Get all unique states from q_3
states_in_q3 = set(state for (state, action) in q_3.keys())

for state in states_in_q3:
    actions = simulator.get_actions(state)
    
    # Find action with highest Q-value for this state
    q_values_for_state = [(action, q_3.get((state, action), 0.0)) for action in actions]
    best_action = max(q_values_for_state, key=lambda x: x[1])[0]
    
    policy_greedy.append({
        'state': state,
        'action': best_action
    })

# Save policy to TSV file
df_policy_greedy = pd.DataFrame(policy_greedy)
df_policy_greedy.to_csv('submission/policy-greedy.tsv', sep='\t', index=False)

print(f"Extracted policy for {len(policy_greedy)} states")
print(f"Sample of policy:")
print(df_policy_greedy.head(10))

Extracted policy for 170 states
Sample of policy:
   state  action
0      1      -1
1      2      -1
2      3      -1
3      4      -1
4      5      -1
5   1030      -1
6      7      -1
7    519      -1
8   1543      -1
9   1031      -1


Submit "policy-greedy.tsv" in Gradescope.

## Part 5: Implement Large Policy

Train a more optimal policy using q-learning.
Save the policy in a file "policy-optimal.tsv" with columns state and action.

Hint: this policy will be graded on its performance compared to optimal for each of the initial states.
**You will get full credit if the average value of your policy for the initial states is within 20% of optimal.**
Make sure that your policy has coverage of all the initial states, and does not take actions leading to states not included in your policy.
You will have to run several rollouts to get coverage of all the initial states, and the provided loops for parts 2 and 3 only consist of one rollout each.

Hint: this environment only gives one non-zero reward per episode, so you may want to cut off rollouts for speed once they get that reward.
But make sure you update the q-values first!

In [44]:
# Define optimal epsilon-greedy policy that will be used during training
def optimal_epsilon_greedy_policy(state, actions):
    epsilon = 0.15  # Lower epsilon for more exploitation
    
    # With probability epsilon, choose a random action (exploration)
    if random.random() < epsilon:
        return random.choice(actions)
    
    # Otherwise, choose the action with highest Q-value (exploitation)
    q_values_for_state = [(action, q_5.get((state, action), 0.0)) for action in actions]
    best_action = max(q_values_for_state, key=lambda x: x[1])[0]
    
    return best_action

In [ ]:
# Train an optimal policy using q-learning
# Run MORE episodes to ensure coverage of all initial states
q_5 = {}

alpha = 1.0  # learning rate
gamma = 0.9  # discount factor

# Run many more episodes to get good coverage
num_episodes = 500

print(f"Training optimal policy with {num_episodes} episodes...")

for episode in range(num_episodes):
    if episode % 100 == 0:
        print(f"Episode {episode}/{num_episodes}, Q-table size: {len(q_5)}")
    
    for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(optimal_epsilon_greedy_policy, max_steps=1024):
        # Get current Q-value (default to 0 if not seen before)
        old_value = q_5.get((curr_state, curr_action), 0.0)
        
        # Calculate max Q-value for next state
        next_actions = simulator.get_actions(next_state)
        max_next_q = max([q_5.get((next_state, a), 0.0) for a in next_actions])
        
        # Q-learning update: Q(s,a) = Q(s,a) + alpha * [r + gamma * max_a' Q(s',a') - Q(s,a)]
        new_value = old_value + alpha * (next_reward + gamma * max_next_q - old_value)
        
        # Update Q-table
        q_5[(curr_state, curr_action)] = new_value
        
        if next_reward > 0:
            # Reached terminal state, cut off this episode for speed
            break

print(f"Training complete! Q-table has {len(q_5)} state-action pairs")

# Extract optimal policy from learned Q-values
policy_optimal = []

# Get all unique states from q_5
states_in_q5 = set(state for (state, action) in q_5.keys())

for state in states_in_q5:
    actions = simulator.get_actions(state)
    
    # Find action with highest Q-value for this state
    q_values_for_state = [(action, q_5.get((state, action), 0.0)) for action in actions]
    best_action = max(q_values_for_state, key=lambda x: x[1])[0]
    
    policy_optimal.append({
        'state': state,
        'action': best_action
    })

# Save optimal policy to TSV file
df_policy_optimal = pd.DataFrame(policy_optimal)
df_policy_optimal.to_csv('submission/policy-optimal.tsv', sep='\t', index=False)

print(f"\nExtracted optimal policy for {len(policy_optimal)} states")
print(f"Number of initial states in simulator: {len(simulator.initial_states)}")
print(f"\nSample of optimal policy:")
print(df_policy_optimal.head(10))

Training optimal policy with 500 episodes...
Episode 0/500, Q-table size: 0
Episode 100/500, Q-table size: 4155
Episode 200/500, Q-table size: 4326
Episode 300/500, Q-table size: 4395
Episode 400/500, Q-table size: 4434
Training complete! Q-table has 4492 state-action pairs

Extracted optimal policy for 1817 states
Number of initial states in simulator: 71

Sample of optimal policy:
   state  action
0      0      -1
1      1      -1
2      2      -1
3      3      -1
4      4      -1
5      5      -1
6      6      -1
7      7      -1
8      9      -1
9     10       1


Submit "policy-optimal.tsv" in Gradescope.

## Part 6: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 7: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.

In [ ]:
with open ('submission/acknowledgements.txt', 'w') as f:
    f.write("None")